#Import management

In [24]:
#we import sys to add the models folder to path, allowing us to use those scripts in visualization
import sys
sys.path.insert(1, '../models/')

import pandas as pd
import numpy as np
import panel as pn
import clean_data as cd

pn.extension('tabulator')

import hvplot.pandas

--Data Management--
Now, we get the .csv files with raw data from the '/data' folder.

In [25]:
data_path = '../data/all_sections.csv'

--File Reading--

We're now going to read the files that we got beforehand.

In [26]:
data = pd.read_csv(data_path)
data.columns

Index(['OrgDefinedId', 'Secciones',
       'Entrega de la mañana D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la tarde D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la mañana (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la tarde (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la mañana (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la tarde (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la mañana (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Entrega de la tarde (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>',
       'Semana 1\Week1 Subtotal Numerator',
       'Semana 1\Week1 Subtotal Denominator',
       'Entrega de la tarde (D5) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\W

...And now we'll call the data cleaning models on this data.

In [27]:
#We rename the columns to make them easier to work with
data.rename(columns={
    'Entrega de la mañana D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'1AM',
    'Entrega de la tarde D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'1PM',
    'Entrega de la mañana (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'2AM',
    'Entrega de la tarde (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'2PM',
    'Entrega de la mañana (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'3AM',
    'Entrega de la tarde (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'3PM',
    'Entrega de la mañana (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'4AM',
    'Entrega de la tarde (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'4PM',
    'Semana 1\Week1 Subtotal Numerator':'week_1_numerator',
    'Semana 1\Week1 Subtotal Denominator':'week_1_denominator',
    'Entrega de la tarde (D5) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'5PM',
    'Entrega de la mañana (D5) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'5AM',
    'Entrega de la mañana (D6) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'6AM',
    'Semana 2\Week 2 Subtotal Numerator':'week_2_numerator',
    'Semana 2\Week 2 Subtotal Denominator':'week_2_denominator',
    'Calculated Final Grade Numerator':'final_numerator',
    'Calculated Final Grade Denominator':'final_denominator'
}, inplace = True)

data.drop(columns={'Adjusted Final Grade Numerator', 'Adjusted Final Grade Denominator','End-of-Line Indicator'}, axis=1, inplace=True)

data.columns

Index(['OrgDefinedId', 'Secciones', '1AM', '1PM', '2AM', '2PM', '3AM', '3PM',
       '4AM', '4PM', 'week_1_numerator', 'week_1_denominator', '5PM', '5AM',
       '6AM', 'week_2_numerator', 'week_2_denominator', 'final_numerator',
       'final_denominator', 'Adjusted Final Grade Numerator',
       'Adjusted Final Grade Denominator', 'End-of-Line Indicator'],
      dtype='object')